## Download data source

* Download the data needed for this jupyter notebook from kaggle and store it in a new folder (the-movies-dataset) in the current directory.


* Upon running this cell, the user will be asked for their username and key which can be found in a fresh api token from kaggle.

* Instructions to get api token to authenticate the data request (Note: kaggle account required):
    1. Sign into kaggle.
    2. Go to the 'Account' tab of your user profile and select 'Create New Token'. 
    3. This will trigger the download of kaggle.json, a file containing your API credentials.

* If the folder has been created and the files are already in that folder, than this cell does nothing and requires no credentials.

* Data Source Information: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv


In [1]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading the-movies-dataset.zip to .\the-movies-dataset


100%|██████████| 228M/228M [00:11<00:00, 21.6MB/s] 


## Combine Raw Data

Combining certain data from the necessary csv files into a single dataframe (complete_df).

* Rows are removed from each dataframe when they do not have sufficient data for a column or the data from a column does not exist.
* This kind of row removal is done before multiple copies of the same movie data becomes present in multiple rows, to save time and space.
* Iteration through rows of a dataframe at this level is inefficient compared to list iteration.
* This is why the dataframes are converted into lists before iteration and then back again to dataframes, so the merge function can be applied to combine the data into a single dataframe (complete_df).

In [5]:
import pandas as pd
import time

start_time = time.time()


pd.set_option('display.max_colwidth', None)

movies_df = pd.read_csv('./the-movies-dataset/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={'genres':"string","id":"string","title": "string", "tagline": "string","overview":"string",
                                    "production_companies" :"string"})[["genres","id" ,"title","tagline", "overview","production_companies"]]
movies_df.dropna(inplace = True)
movies_lst = [row for row in movies_df.values.tolist() if not (row[0][len(row[0])  - 2:] == "[]" or row[5][len(row[5]) - 2:] == "[]")]
movies_df = pd.DataFrame(movies_lst, columns = ("genres","id" ,"title","tagline", "overview","production_companies"), dtype = str)



ratings_df = pd.read_csv('./the-movies-dataset/ratings.csv', usecols = ("userId", "movieId", "rating"),
                       dtype={"userId": "string","movieId": "string","rating": "string"})[["userId", "movieId", "rating"]]
ratings_df.rename(columns={"movieId": "id"}, inplace = True)
ratings_df.dropna(inplace = True)


# Question: What if the removal of duplicate movie ids per user was processed here instead of the cell below???
# Answer: The duplicate removal function can be ran here,...
# but the complete_list in the cell below can also be iterated over with relative complexity in order to remove duplicates.
# The iteration in the next cell also populates the gap list...
# which is critical to be ran directly before the function that determines bounds for users rated movies.
# So, omitting the no duplicate function in this cell and making it run in the next cell avoids redundant iteration.


# Question: What if the test and train ratings bounds was enforced here instead of the cell below???
# Answer: The merge functions below needs to be executed before determining test and train users, because merge will remove rows and ratings from users...
# before enforcing the users to be in a certain bounds for the number of their ratings. 
# The current timing of this function will ensure that the final users are within the set train or test bounds.


keywords_df = pd.read_csv('./the-movies-dataset/keywords.csv', usecols = ("id", "keywords"), dtype={"id": "string","keywords":"string"})[["id", "keywords"]]
keywords_df.dropna(inplace = True)
keywords_lst = [row for row in keywords_df.values.tolist() if not (row[1][len(row[1])  - 2:] == "[]")]
keywords_df = pd.DataFrame(keywords_lst, columns = ("id", "keywords"), dtype = str)


credits_df = pd.read_csv("./the-movies-dataset/credits.csv", usecols = ("cast", "id"), dtype={"cast": "string", "id": "string"})[["cast", "id"]]
credits_df.dropna(inplace = True)
credits_lst = [row for row in credits_df.values.tolist() if (not row[0][len(row[0])  - 2:] == "[]")]
credits_df = pd.DataFrame(credits_lst, columns = ("cast", "id"), dtype = str)


# Default merge is inner: This only keeps movies that have the id existing in both dataframes.
complete_df =  pd.merge(movies_df, ratings_df, on ="id")
complete_df =  pd.merge(complete_df,keywords_df, on ="id")
complete_df  = pd.merge(complete_df,credits_df, on ="id")


complete_df.sort_values(by = 'userId', inplace = True)


# Master dataframe: For each (user id, movie id) row combination there is the combined movie data from movies_df, ratings_df, keywords_df, and credits_df for the movie id in question.
# The columns are reordered.
complete_df  = complete_df.loc[:,['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview" ]]

# For testing:
print("Minutes taken:", (time.time()-start_time)/60)

# Notice: With the movies, keywords, and credits dataframes, list conversion happens before dropping empty entries
# Tested on personal machine:
# tested without list conversion (old code): 1 minute and 5.7 seconds
# tested with list conversion (current code): 37.1 seconds

Minutes taken: 0.6039181590080261


## User Selection and Data Extraction

1. Remove duplicate movies rated by the same user
2. Randomly choose users that fall into the appropriate bounds for the number of ratings to be a svd user, train user, or test user
3. Extract the data from those users and structure it into a list to be written too a csv file

In [6]:
import ast
import time
from numpy.random import Generator, PCG64

def populate_names(item):
    """Extract names from the syntax of certain data entries:"""
    string  = item[1:-1]
    jsons = string.split("}, ")   
    names = ""
    index = 0
    for item in jsons:
        if(index == len(jsons)-1):
            temp_dict = ast.literal_eval(item)
            names+=str(temp_dict["name"])
        else:
            temp_dict = ast.literal_eval(item+"}")
            names+=str(str(temp_dict["name"])+" ")
        index += 1
    return names


def provide_data(row):
    """Extract data from row of complete_list:"""
    movie_data = []
    movie_data.append(int(row[0]))
    movie_data.append(int(row[1]))
    movie_data.append(float(row[2]))
    movie_data.append(row[3])  

    movie_data.append(populate_names(row[4]))
    movie_data.append(populate_names(row[5]))
    movie_data.append(populate_names(row[6]))
    movie_data.append(populate_names(row[7]))

    movie_data.append(str(row[8]))
    movie_data.append(str(row[9]))
    return movie_data
    


# main:
start_time = time.time()

SEED_INT = 42
outer_gen = Generator(PCG64(SEED_INT))
# The list of rows with users id, the users rating for the movie, and metadata for the movie:
# Note: It is sorted by user_id.
complete_list = complete_df.values.tolist()

print("Complete number of users:", len(list(complete_df["userId"].unique()))) # 260788

# The same as complete_list where data is omitted for movies that have already been rated by the user in a previous row
complete_list_no_dups = []

# Distinguish the user the row belongs to:
last_id = complete_list[0][0]

# The set of movies that a user has rated:
# It is used to omit later ratings of a movie that the user has already rated.
movie_set = set()

# The number of rows of movie data a single user takes up for each user:
gaps = []

# Appended to gaps when all of a users rows of movie data have been counted:
gap_len = 0


# Populates gaps and complete_list_no_dups by omitting movies that already have a rating in respect to each user:
# Note: This code is faster than using dataframe methods.
# Example dataframe method: Filter data by user and then remove duplicate movie ids for each user.
# This avoids slow dataframe iteration, but the filter method is also slow.
for row in complete_list:
    if last_id != row[0]:
        movie_set= set()
        complete_list_no_dups.append(row)
        movie_set.add(row[1])
        gaps.append(gap_len)
        gap_len = 1
    else:
        if row[1] not in movie_set:
            complete_list_no_dups.append(row)
            gap_len+=1
            movie_set.add(row[1])
    last_id = row[0]

# Add the last gap_len:
gaps.append(gap_len)



# Bounds represents the first index and last index(non inclusive) of the range of ratings for a user in the sorted complete_list_no_dups
full_index = 0 
bounds = [] 

for user_index in range(len(gaps)):
    bounds.append([full_index, full_index+gaps[user_index]])
    full_index+=gaps[user_index]    
 



# These set the rating requirements for svd, train, and test users:
SVD_USER_RATING_LB = 20
SVD_USER_RATING_UB = 30
USER_RATING_LB = 5
USER_RATING_UB = 10

# Makes selection of user bounds random:
outer_gen.shuffle(bounds)

NOF_SVD_USERS = 10000
NOF_TRAIN_USERS = 10000
NOF_TEST_USERS = 10000


last_index = -1
bounds_svd_users = []
bounds_train_users = []
bounds_test_users = []


index = 0
for item in bounds:
    if item[1]-item[0] >=SVD_USER_RATING_LB and item[1]-item[0] <=SVD_USER_RATING_UB:
        bounds_svd_users.append(item)
        if len(bounds_svd_users) == NOF_SVD_USERS:
            last_index = index
            print("nof svd users met")
            break
    index+=1



index+=1
for item in bounds[last_index:]:
    if item[1]-item[0] >=USER_RATING_LB and item[1]-item[0] <=USER_RATING_UB:
        bounds_train_users.append(item)
        if len(bounds_train_users) == NOF_TRAIN_USERS:
            last_index = index
            print("nof train users met")
            break
    index+=1

index+=1
for item in bounds[last_index:]:
    if item[1]-item[0] >=USER_RATING_LB and item[1]-item[0] <=USER_RATING_UB:
        bounds_test_users.append(item)
        if len(bounds_test_users) == NOF_TEST_USERS:
            print("nof test users met")
            break



# Sample the data from complete_list_no_dups once the bounds (low memory) have been randomly selected:
sampled_data = []


for bound in bounds_svd_users:
    for movie in complete_list_no_dups[bound[0]:bound[1]]:
        movie_data = provide_data(movie)
        sampled_data.append(movie_data)


for bound in bounds_train_users:
    for movie in complete_list_no_dups[bound[0]:bound[1]]:
        movie_data = provide_data(movie)
        sampled_data.append(movie_data)



for bound in bounds_test_users:
    for movie in complete_list_no_dups[bound[0]:bound[1]]:
        movie_data = provide_data(movie)
        sampled_data.append(movie_data)



print("Minutes taken:", (time.time()-start_time)/60)


Complete number of users: 260788
nof svd users met
nof train users met
nof test users met
Minutes taken: 6.1188988049825035


## Write Data

Save selected data in constructed_data.csv file so that cells below it can run without running this cell and above.


In [7]:
import csv
import os

current_directory = os.getcwd()
final_directory = os.path.join(current_directory, 'constructed_data')
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

output_path = os.path.join("constructed_data", "constructed_data.csv")

with open(output_path, "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview"])
    writer.writerows(sampled_data)

## Read Data
This is the starting cell to run if the data is already saved to the constructed_data.csv. 

In [1]:
import csv
import os

data_list =[]


input_path = os.path.join("constructed_data", "constructed_data.csv")

with open(input_path, 'r', encoding="utf-8") as f:
    csv_reader = csv.reader(f)
    data_list = list(csv_reader)

data_list = data_list[1:]

## Organize Data:

Extract data from data_list into user_to_data_svd, user_to_data_train, and user_to_data_test.

Each of these can be thought of a list of users and each user can be though of a list of movies ratings by the user
 and the movie data for that movie.

In [2]:
NOF_SVD_USERS = 10000
NOF_TRAIN_USERS = 10000
NOF_TEST_USERS = 10000


user_to_data_svd = []
user_to_data_train= []
user_to_data_test = []

user_id = data_list[0][0]
ratings = []
user_index = 0



for row in data_list:
    if (row[0]!=user_id):
        if(user_index<NOF_SVD_USERS and user_index>=0):
            user_to_data_svd.append(ratings)
        elif(user_index<NOF_TRAIN_USERS+NOF_TRAIN_USERS and user_index>=NOF_SVD_USERS):
            user_to_data_train.append(ratings)
        else:
            user_to_data_test.append(ratings)         
        user_id = row[0]
        ratings = [row]
        user_index+=1
    else:
        ratings.append(row)



user_to_data_test.append(ratings)

## Create Features and Target Ratings

* This cell populates input features and target ratings for train and test users.

* Each train and test user has values for three features (feature 1,2, and 3) and a target rating from one randomly selected movie that they watched.

* feature_1 and feature_2 are un_weighted and weighted averages respectively of the users non-target movies and serve as predictions to the target movie rating.

* feature_3 is another rating prediction of the target movie by an svd model using the best hyperparameters found in the bayesian_optimization.ipynb notebook.

* Features 1, 2, and 3 are processed and eventually become model parameters to the linear model in the final cell. 

* The parameters for the train users are used to train a linear model and the parameters for the test users are used to test a linear model.

In [3]:
import spacy
spacy.cli.download("en_core_web_sm")
from ordered_set import OrderedSet
from collections import Counter
import numpy as np
from numpy.random import Generator, PCG64
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import mean_squared_error
import time
from numba import njit
import copy
from dask.distributed import Client, LocalCluster
import dask.array as da
import os



def re_index(user_to_data_temp, old_to_new, last_index, cnt):
    """
    Continue to re-index the user ids and the movie ids for the test or train users in the order of their occurrence...
    starting at the point where the svd users left off. 
    """
    for user in user_to_data_temp:
        for movie in user: 
            if(movie[1] in old_to_new.keys()):
                movie[1] = old_to_new[movie[1]]
            else:
                old_to_new[movie[1]] = last_index
                movie[1] = last_index
                last_index+=1      
            movie[0] = cnt
        cnt+=1



def populate(gen_input, overall_average, cnt, user_to_data_temp,
                 feature_1, feature_2, list, target_rating, rating_to_predict, movies_order):
    """
    Populate feature 1, feature 2, and target ratings while prepping variables to be used to populate feature 3.
    """
    for user in user_to_data_temp:
        # randomly chooses a movie to be the target movie:
        rand_num  = gen_input.integers(0, len(user))
        index = 0

        # used to build feature_1:
        user_rating_sum = 0
        usr_rating_count = 0

        # used to build feature_2:
        words = OrderedSet()
        word_count_dict_per_movie = []
        target_movie_word_counts_dict = dict()
        non_target_ratings = []

        for movie in user:
            movies_order.add(movie[1])

            # corpus extraction:
            movie_string = ""
            for j in range (3,len(movie)):
                if(j!= len(movie)-1):
                    movie_string+= movie[j]+" "
                else:
                    movie_string+= movie[j]

            doc = nlp(movie_string)
            filtered_tokens = [token.lemma_ for token in doc if not token.is_stop]

            words.update(filtered_tokens)

            if(index == rand_num):
                # target movies logic:
                rating_to_predict.append([int(movie[0]), int(movie[1])])
                target_rating.append(float(movie[2]))

                target_movie_word_counts_dict = Counter(filtered_tokens)
            else:
                # non-target movie logic:
                overall_average+=float(movie[2])
                cnt += 1

                user_rating_sum+=float(movie[2])
                usr_rating_count +=1

                list.append([int(movie[0]), int(movie[1]), float(movie[2])])
                non_target_ratings.append(float(movie[2]))

                word_count_dict_per_movie.append(Counter(filtered_tokens))

            index+=1

        # build feature_1:
        feature_1.append(user_rating_sum/usr_rating_count)

        # preprocessing for feature_2:
        word_counts_in_order_per_movie = [[temp_dict[movie] if movie in temp_dict.keys() else 0 for movie in words] for temp_dict in word_count_dict_per_movie]
        target_word_counts_in_order = [target_movie_word_counts_dict[movie] if movie in target_movie_word_counts_dict.keys() else 0 for movie in words]
        complete_word_counts = word_counts_in_order_per_movie.copy()
        complete_word_counts.append(target_word_counts_in_order)
        transformed_word_counts = TfidfTransformer().fit_transform(complete_word_counts).toarray()
        cosine_sim = linear_kernel(X = transformed_word_counts[0:-1],Y = [transformed_word_counts[-1]])
        cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))
        numerator = 0
        denominator = 0

        # create weighted average based on rating and cosine similarity between the target and the rated movies normalized Tf-idf vectors:
        for i in range(len(non_target_ratings)):
            numerator += cosine_sim[i]*non_target_ratings[i]
            denominator += cosine_sim[i]

        # build feature_2 (handle undefined case):
        if denominator == 0:
            feature_2.append(feature_1[-1])
        else:
            feature_2.append(numerator/denominator)

    overall_average = overall_average/cnt

    gen_input.shuffle(list)

    return overall_average



@njit
def epoch(list, b1, b2, p, q, overall_average, lr, rt):
    """
    Update the parameters (b1, b2, q, and p) for each row in the list using stochastic gradient descent.
    """
    for row in list:
        u = int(row[0])
        i = int(row[1])
        r = row[2]

        pred = overall_average+b1[u]+b2[i]+np.dot(p[u],q[i])
        error = r-pred
        b1[u] += lr*(error- rt*b1[u])
        b2[i] += lr*(error- rt*b2[i])
        temp = lr*(error*q[i] -rt*p[u])
        q[i] += lr*(error*p[u] -rt*q[i])
        p[u] += temp



def svd_iterative(gen_input, list, n, epochs, rt, lr, overall_average, nof_users, nof_movies):
    """
    An iterative SVD method that has been shown to out perform non-iterative svd methods
    """
    
    q = gen_input.normal(0, .1, (nof_movies, n))
    p = gen_input.normal(0, .1, (nof_users, n))


    b1 = np.zeros(nof_users)
    b2 = np.zeros(nof_movies)

    np_array = np.array(list)

    for _ in range(epochs):
        epoch(np_array, b1, b2, p, q, overall_average, lr, rt)

    return b1, b2, p, q


def generate_param_block(block):
    """
    This function computes feature_1_train_temp, feature_1_test_temp, feature_2_train_temp, feature_2_test_temp,...
    feature_3_train_temp, feature_3_test_temp, target_rating_train_temp, and target_rating_test_temp... 
    for each row of the block then adds them to the output block, which is returned after all the rows in the block have been processed.
    """
    output_block = []

    for row in block:
        feature_1_train_temp = []
        feature_1_test_temp= []
        feature_2_train_temp= [] 
        feature_2_test_temp= []
        feature_3_train_temp= [] 
        feature_3_test_temp= [] 
        target_rating_train_temp= [] 
        target_rating_test_temp= []


        input_seed, user_to_data_svd_temp, user_to_data_train_temp, user_to_data_test_temp, nof_latent_features, epochs, rt, lr = row

        gen = Generator(PCG64(input_seed))


        # re-index the user ids and the movie ids in the order of their occurrence:
        old_to_new_svd  = dict()
        last_index_svd = 0
        svd_cnt = 0

        for user in user_to_data_svd_temp:
            for movie in user: 
                if(movie[1] in old_to_new_svd.keys()):
                    movie[1] = old_to_new_svd[movie[1]]
                else:
                    old_to_new_svd[movie[1]] = last_index_svd
                    movie[1] = last_index_svd
                    last_index_svd+=1      
                movie[0] = svd_cnt
            svd_cnt+=1

        old_to_new_train = copy.deepcopy(old_to_new_svd)
        old_to_new_test = copy.deepcopy(old_to_new_svd)
        re_index(user_to_data_train_temp, old_to_new_train, last_index_svd, svd_cnt)
        re_index(user_to_data_test_temp, old_to_new_test, last_index_svd, svd_cnt)


        # add the svd users to the train_list and test_list...
        # (svd users contribute to training an svd model without target ratings to be predicted by an svd model)
        train_list = []
        test_list = []

        movies_order_svd = set()
        overall_average_svd = 0 
        cnt_svd = 0

        for user in user_to_data_svd_temp:
            for movie in user:
                movies_order_svd.add(movie[1])
                train_list.append([int(movie[0]), int(movie[1]), float(movie[2])])
                test_list.append([int(movie[0]), int(movie[1]), float(movie[2])])
                overall_average_svd+=float(movie[2])
                cnt_svd += 1

        # Populate a collection of variables including feature_1_train_temp, feature_2_train_temp, feature_1_test_temp, feature_2_test_temp...
        # target_ratings, and variables that are required by the svd method to populate feature 3.
        # The train users are added to the train_list and the test users are added to the test_list. These provide users that...
        # train their respective svd models, but also have a single rating to be predicted by their respective svd model.
                
        train_ratings_to_predict = []
        test_ratings_to_predict = []  
        movies_order_train = copy.deepcopy(movies_order_svd)
        movies_order_test = copy.deepcopy(movies_order_svd)        
        overall_average_train = populate(gen,overall_average_svd, cnt_svd, user_to_data_train_temp,
                    feature_1_train_temp, feature_2_train_temp, train_list, target_rating_train_temp, train_ratings_to_predict, movies_order_train)
        overall_average_test = populate(gen, overall_average_svd, cnt_svd, user_to_data_test_temp,
                    feature_1_test_temp, feature_2_test_temp, test_list, target_rating_test_temp, test_ratings_to_predict, movies_order_test)


        # build feature_3_train and feature_3_test with svd algorithm:
        b1, b2, p, q = svd_iterative(gen, train_list, nof_latent_features, epochs, rt, lr,
                                    overall_average_train, len(user_to_data_svd_temp)+len(user_to_data_train_temp), len(movies_order_train))

        feature_3_train_temp = ([overall_average_train + b1[pair[0]]+b2[pair[1]]
                                    +np.dot(p[pair[0]],q[pair[1]]) for pair in train_ratings_to_predict])
        
        b1, b2, p, q = svd_iterative(gen, test_list, nof_latent_features, epochs, rt, lr,
                                    overall_average_test, len(user_to_data_svd_temp)+len(user_to_data_test_temp), len(movies_order_test))

        feature_3_test_temp = ([overall_average_test + b1[pair[0]]+b2[pair[1]]
                                    +np.dot(p[pair[0]],q[pair[1]]) for pair in test_ratings_to_predict])

        arr = np.array([feature_1_train_temp, feature_1_test_temp, feature_2_train_temp, feature_2_test_temp,
        feature_3_train_temp, feature_3_test_temp, target_rating_train_temp, target_rating_test_temp])
        
        arr = arr.transpose()
        output_block.append(arr)


    return np.array(output_block, dtype = "float32")

# main:
start_time = time.time()

cluster = LocalCluster(n_workers=os.cpu_count())
client = Client(cluster)

ITERATIONS = 40

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# seed for consistent results across runtimes:
SEED_INT = 10
outer_gen = Generator(PCG64(SEED_INT))

# These are variables to construct the final model:

# user average predictions:
feature_1_train = [] 
feature_1_test = [] 

# weighted user average predictions:
feature_2_train = [] 
feature_2_test = []

# svd predictions:
feature_3_train = [] 
feature_3_test = [] 

# all the users real ratings:
target_rating_train = []
target_rating_test = []

# hyperparameters tuned by the bayesian_optimization.ipynb notebook:
nof_svd_users, nof_train_users, nof_latent_features, epochs, rt, lr = (467, 189, 292, 268, 0.01588566520994121, 0.04980345674001443)

nof_test_users = nof_train_users

parameters_list = []


# Note: 
# This loop has a high cost because it makes a new choice of users every iteration.
# However, if the choice of users was repeated for a number of iterations before switching then there would be more noise.
# The more variety of users the less noise. Also, the cost is relatively low compared to the rest of the operations.

# Note:
# user_to_data_svd_copy and user_to_data_train_copy are converted from list to numpy array of objects, then sampled, and then converted back to a list. 
# This is to avoid using more than one random type and suppresses the warning about arrays being a ragged nested sequences. 

for _ in range(ITERATIONS):
    parameters_list.append([outer_gen.integers(0,100000),
                            list(copy.deepcopy(outer_gen.choice(np.array(user_to_data_svd, dtype='object'), nof_svd_users, replace = False))),
                            list(copy.deepcopy(outer_gen.choice(np.array(user_to_data_train, dtype = "object"), nof_train_users, replace = False))),
                            list(copy.deepcopy(outer_gen.choice(np.array(user_to_data_test, dtype='object'), nof_test_users, replace = False))),
                            nof_latent_features, epochs, rt, lr])



parameters_arr = np.array(parameters_list, dtype="object")
dask_array = da.from_array(parameters_arr, chunks=(int(ITERATIONS/8),8))
results = dask_array.map_blocks(generate_param_block, chunks = (int(ITERATIONS/8), nof_train_users,8), dtype="float32").compute()




for block in results:
    for row in block:
        feature_1_train.append(row[0]) 
        feature_1_test.append(row[1]) 
        feature_2_train.append(row[2]) 
        feature_2_test.append(row[3]) 
        feature_3_train.append(row[4])  
        feature_3_test.append(row[5])  
        target_rating_train.append(row[6]) 
        target_rating_test.append(row[7]) 



print("Feature 1:")
print("RMSE with train users:", mean_squared_error(target_rating_train, feature_1_train, squared = False))
print("RMSE with test users:", mean_squared_error(target_rating_test, feature_1_test, squared = False))

print("Feature 2:")
print("RMSE with train users:", mean_squared_error(target_rating_train, feature_2_train, squared = False))
print("RMSE with test users:", mean_squared_error(target_rating_test, feature_2_test, squared = False))

print("Feature 3:")
print("RMSE with train users:", mean_squared_error(target_rating_train, feature_3_train, squared = False))
print("RMSE with test users:", mean_squared_error(target_rating_test, feature_3_test, squared = False))


client.close()
cluster.close()


print("Minutes taken:", (time.time()- start_time)/60)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


d:\repos\movie_rec_proj\venv\lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 309.02 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Feature 1:
RMSE with train users: 1.1058986
RMSE with test users: 1.1079229
Feature 2:
RMSE with train users: 1.120416
RMSE with test users: 1.1226492
Feature 3:
RMSE with train users: 1.0397173
RMSE with test users: 1.0418123
Minutes taken: 6.212504545847575


## Final Model Creation

The features and the target ratings constructed above are used to train and test a model.


Note: Each feature can function as a predictor to the target movies rating on their own. The purpose of the model (loosely speaking) is how much weight to give to each feature for the optimal prediction.

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import time


def train_and_test(train_inputs, test_inputs):
    """Build, train, and test a model, then return the performance metric."""

    # linear regression model:
    reg = LinearRegression()

    # train model:
    reg.fit(train_inputs, target_rating_train)

    # make predictions for test inputs:
    predictions = reg.predict(test_inputs)

    return(mean_squared_error(target_rating_test, predictions, squared=False))


def average_results(nof_runs, train_inputs, test_inputs):
    """Average the performance results for a number of models with identical inputs."""
    mse_sum=0
    for _ in range(nof_runs):
        mse_sum+= train_and_test(train_inputs, test_inputs)
    return mse_sum/nof_runs


def test_parameters(nof_runs, train_input_features, test_input_features):
    """Test_parameters for a number of runs and return performance results."""
    train_inputs = [list(pair) for pair in train_input_features]
    test_inputs = [list(pair) for pair in test_input_features]
    # Note: nof_runs only applies when randomness is a factor of the model.
    return average_results(nof_runs, train_inputs, test_inputs)


# main:

start_time  = time.time()

# Note: The features are inputs so any features can be removed from the model if necessary as long as there is at least a single feature.
# target_rating_train and target_rating_test are required and therefore are accessed externally.

# Note: nof_runs only applies when randomness is a factor of the model.
print("Average RMSE test score:", test_parameters(1, zip(feature_1_train,feature_2_train, feature_3_train), zip(feature_1_test,feature_2_test, feature_3_test)))


print("Minutes taken:", (time.time()-start_time)/60)

Average RMSE test score: 1.0372942686080933
Minutes taken: 0.000166626771291097
